<a href="https://colab.research.google.com/github/gracemarshall/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version 

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-27 23:59:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-02-27 23:59:38 (11.0 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [5]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [6]:
# Create the vine_table. DataFrame
vine_table_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_table_df.show()



+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3VR960AHLFKDV|          4|            0|          0|   N|                Y|
|R16LGVMFKIUT0G|          5|            0|          0|   N|                Y|
|R1AIMEEPYHMOE4|          5|            1|          1|   N|                Y|
|R1892CCSZWZ9SR|          3|            0|          0|   N|                Y|
|R285P679YWVKD1|          3|            0|          0|   N|                N|
| RLB33HJBXHZHU|          5|            0|          0|   N|                Y|
|R1VGTZ94DBAD6A|          5|            2|          2|   N|                Y|
|R168KF82ICSOHD|          5|            0|          0|   N|                Y|
|R20DIYIJ0OCMOG|          5|            0|          0|   N|                Y|
| RD46RNVOHNZSC|          5|            0|          0|   N|     

In [7]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
vine_fil_df = vine_table_df.filter('total_votes>=20')
vine_fil_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|
|R3GNSIFV1J2Y2B|          1|           15|         60|   N|                N|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|
|R3OFB4P7Y8WR27|          1|           15|         26|   N|                Y|
|R3MTAYGQM25N63|          4|           58|         59|   N|     

In [8]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
import pyspark.sql.functions as F
helpful_df = vine_fil_df.withColumn('result_column',((F.col("helpful_votes") / F.col("total_votes"))*100)).filter('result_column>50')
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     result_column|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y| 92.12121212121212|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y| 91.30434782608695|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|             100.0|
| RR99CPG695T0I|          5|          215|        248|   N|                N| 86.69354838709677|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y| 97.72727272727273|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y| 96.15384615384616|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y| 96.29629629629629|
|R3OFB4P7Y8WR27|          1|  

In [9]:
yes_review_df = helpful_df.filter(helpful_df.vine == 'Y')
yes_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    result_column|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R2BQOD1R0228FN|          3|           17|         26|   Y|                N|65.38461538461539|
| RC31RUPFOHBHQ|          4|          102|        117|   Y|                N|87.17948717948718|
| REN3N1WITLF1Y|          5|           33|         37|   Y|                N| 89.1891891891892|
| R71RZQ9UZVG47|          4|           38|         47|   Y|                N|80.85106382978722|
|R38NMQBH88HLM6|          4|           18|         24|   Y|                N|             75.0|
|R33FGX9EE3QVR6|          4|           26|         26|   Y|                N|            100.0|
|R1KIOIK6WEYE59|          3|           19|         20|   Y|                N|             95.0|
|R25X9BMOB3FD0E|          4|           3

In [10]:
no_review_df = helpful_df.filter(helpful_df.vine == 'N')
no_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     result_column|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y| 92.12121212121212|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y| 91.30434782608695|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|             100.0|
| RR99CPG695T0I|          5|          215|        248|   N|                N| 86.69354838709677|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y| 97.72727272727273|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y| 96.15384615384616|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y| 96.29629629629629|
|R3OFB4P7Y8WR27|          1|  

In [11]:
total_paid_review = yes_review_df.select('vine').count()
total_paid_review

136

In [12]:
paid_five_star = yes_review_df.filter('star_rating == 5').count()
paid_five_star 

74

In [13]:
percent_paid_five_star = (paid_five_star /total_paid_review )*100
percent_paid_five_star

54.41176470588235

In [14]:
unpaid_total_review = no_review_df.select('vine').count()
unpaid_total_review

17999

In [15]:
unpaid_five_star = no_review_df.filter('star_rating == 5').count()
unpaid_five_star

8480

In [16]:
percent_unpaid_five_star = (unpaid_five_star /unpaid_total_review )*100
percent_unpaid_five_star

47.11372854047447